# DoorHinge joint inspector
The `DoorHinge` class is a `ForceElement`, which models a revolute door hinge joint that could exhibits different force/torque characterisitcs at different states due to the existence of different type of torques on the joint. This class implements a "christmas tree" accumulation of these different torques (i.e. function approximation) in an empirical and unprincipled way. Specifically, different curves are assigned to different torques to mimic their evolution based on the joint state and some prespecified parameters. More details can be referred to the doxygen documentation of the `DoorHinge` class.


This tutorial provides a small tool to allow you to play with the `DoorHingeConfig` parameter and visualize the properties of a particular setup. It could facilitate you get the specific kind of door hinge joint behavior you are interested. In this tutorial, we provide a common dishwasher door modeling as an example.

A common dishwasher door has a frictional torque sufficient for it to rest motionless at any angle, a catch at the top to hold it in place, a dashpot (viscous friction source) to prevent it from swinging too fast, and a spring to counteract some of its mass. Two figures are provided to illustrate the dishwasher door hinge torque with the given default parameters. We first used a force gauge to estimate these values roughly. Then we simulate the dishwasher door for minor adjustment.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pydrake.multibody.plant import MultibodyPlant
from pydrake.multibody.tree import (
    DoorHingeConfig, DoorHinge, RevoluteJoint, SpatialInertia
)

# Build a simple MultibodyPlant that contains two bodies and a revolute joint.
# Then add a DoorHinge ForceElement to the joint.
plant = MultibodyPlant(0.0)
spatial_inertia = SpatialInertia()

body_a = plant.AddRigidBody(name="body_a", M_BBo_B=spatial_inertia)
body_b = plant.AddRigidBody(name="body_b", M_BBo_B=spatial_inertia)

revolute_joint = plant.AddJoint(RevoluteJoint(
        name="revolve_joint", frame_on_parent=body_a.body_frame(),
        frame_on_child=body_b.body_frame(), axis=[0, 0, 1],
        damping=0.0))

# Create a default DoorHingeConfig. The user should change the parameters before
# calling AddForceElement.
door_hinge_config = DoorHingeConfig()
door_hinge_config.spring_constant = 3.0

# Add the DoorHinge and finalize the plant.
door_hinge = plant.AddForceElement(DoorHinge(
            joint=revolute_joint, config=door_hinge_config))
plant.Finalize()


In [ ]:
# Prepare the data for plotting.
min_q = 0.0
max_q = np.deg2rad(80)
min_v = -0.1
max_v = 0.1
step_size = 0.0005
q_array = np.arange(min_q, max_q, step_size)
v_array = np.arange(min_v, max_v, step_size)

# Plot the torque over angle at zero velocity
torque_zero_velocity = [
    door_hinge.CalcHingeTorque(q, 0.0)
    for q in q_array
]

# Plot the torque over the angular velocity at 30 degree angle
torque_30_angle = [
    door_hinge.CalcHingeTorque(np.deg2rad(30), v)
    for v in v_array
]
plt.plot(v_array, torque_30_angle)
plt.show()

The first figure shows the static characteristic of the dishwasher door. At $q = 0^{\circ}$, there exists a negative catch torque to prevent the door from moving. After that, the torsional spring torque will dominate to compensate part of the door gravity. The second figure shows the dynamic feature of the dishwasher door at $q = 30^{\circ}$. Whenever the door tries to move, there will be a counter torque to prevent that movement, which therefore keeps the door at rest. Note that, due to the gravity, the dishwasher door will be fully open eventually. However, this process can be really slow because of the default `motion_threshold` is set to be very small.